In [1]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
import pandas as pd
import numpy as np
import string
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split, Dataset
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gc
import json

In [21]:
train = pd.read_pickle('Preprocessing_Train.pkl')

with open('data/strugg.txt', 'w') as f:
    for i in range(len(train)):
        f.write(str(train['label'][i])+'\t'+train['document_text'][i]+'\n')

%run augment.py --input=data/strugg.txt --num_aug=3

f1 = open('data/eda_strugg.txt')
prep = list(f1)
text = [(str(re.split('\t', i)[0]), re.split('\t', i)[1][:-1]) for i in prep]
ex = pd.DataFrame(text, columns=['label', 'text'])

In [23]:
# ex.to_csv('extra.csv')
ex = pd.read_csv('extra.csv')

lab = []
for i in ex.label.apply(json.loads):
    if 8 in i:
        x1 = 1
    else:
        x1 = 0
    if 22 in i:
        x2 = 1
    else:
        x2 = 0
    if 35 in i:
        x3 = 1
    else:
        x3 = 0
    lab.append(np.array([x1,x2,x3]))

label_final = lab+list(train.label.apply(lambda x: x[[8, 22, 35]]))
text_final = list(ex.text)+list(train.document_text)

X_train, X_test, y_train, y_test = train_test_split(text_final, label_final, test_size=0.25)

In [78]:
json.loads('[1,2,3]')

[1, 2, 3]

In [57]:
X_train, X_test, y_train, y_test = train_test_split(train.document_text, train.label_idx, test_size=0.25)
sp_idx = [i for i in range(len(X_train)) if (8 in list(y_train)[i]) or
 (22 in list(y_train)[i]) or (35 in list(y_train)[i])]

with open('data/train.txt', 'w') as f:
    for i in sp_idx:
        f.write(str(list(y_train)[i])+'\t'+list(X_train)[i]+'\n')

%run augment.py --input=data/train.txt --num_aug=3

f1 = open('data/eda_train.txt')
prep = list(f1)
text = [(str(re.split('\t', i)[0]), re.split('\t', i)[1][:-1]) for i in prep]
ex_train = pd.DataFrame(text, columns=['label', 'text'])

generated augmented sentences with eda for data/train.txt to data\eda_train.txt with num_aug=3


In [62]:
lab_t = []
for i in ex_train.label.apply(json.loads):
    if 8 in i:
        x1 = 1
    else:
        x1 = 0
    if 22 in i:
        x2 = 1
    else:
        x2 = 0
    if 35 in i:
        x3 = 1
    else:
        x3 = 0
    lab_t.append(np.array([x1,x2,x3]))

text_t = list(ex_train.text)+list(X_train)

for i in y_train:
    if 8 in i:
        x1 = 1
    else:
        x1 = 0
    if 22 in i:
        x2 = 1
    else:
        x2 = 0
    if 35 in i:
        x3 = 1
    else:
        x3 = 0
    lab_t.append(np.array([x1,x2,x3]))

In [69]:
stop_words = stopwords.words('english')
non_neg_stop = [i for i in stop_words if "n't" not in i and "no" not in i]
lemma = WordNetLemmatizer()


def pos(x):
    if x.startswith('J'):
        return 'a'
    elif x.startswith('V'):
        return 'v'
    elif x.startswith('R'):
        return 'r'
    else:
        return 'n'
    
    
def tok(doc, non_neg=False, lemmatized=False):
    
    if non_neg:
        stop = non_neg_stop
    else:
        stop = stop_words
       
    tokens = re.split(r'\s', doc.lower())
    
    if lemmatized:
        pos_list = nltk.pos_tag(tokens)
        tokens = list(map(lambda x: lemma.lemmatize(x[0], pos(x[1])), pos_list))    

    tokens = [i for i in tokens if i not in stop]
    tokens = [i for i in tokens if len(re.findall(r'\w', i)) >= 2]
    tokens = [re.findall(r"\w[a-zA-Z0-9.-]*\w", i)[0] for i in tokens if re.findall(r"\w[a-zA-Z0-9.-]*\w", i)]
    
    return tokens


def get_voc_new(X, non_neg=True, lemmatized=True):
    
    voc = []
    token_list = []
    for i in X:
        tokens = tok(i, non_neg, lemmatized)
        token_list.append(tokens)
        voc += tokens
        
    voc = list(set(voc))
    
    return voc, token_list

In [70]:
voc, tk_list_train = get_voc_new(text_t, False, True)
# voc, tk_list_train = get_voc_new(train.document_text, False, True)
voc_test, tk_list_test = get_voc_new(X_test, False, True)
voc_dic = {item: idx+1 for idx, item in enumerate(voc)}

In [71]:
def word2idx(token_list):
    
    token_idx = []
    for sent in token_list:
        sent_list = []
        for token in sent:
            if token in voc_dic:
                idx = voc_dic[token]
            else:
                idx = 0
            sent_list.append(idx)
        token_idx.append(sent_list)

    return token_idx


tk_idx_train = word2idx(tk_list_train)
tk_idx_test = word2idx(tk_list_test)

In [72]:
tes = []
for i in y_test:
    if 8 in i:
        x1 = 1
    else:
        x1 = 0
    if 22 in i:
        x2 = 1
    else:
        x2 = 0
    if 35 in i:
        x3 = 1
    else:
        x3 = 0
    tes.append(np.array([x1,x2,x3]))

In [73]:
class dataset_(Dataset):
    
    def __init__(self, X, y, max_len=1024):
        super(dataset_, self).__init__()
        pad = [i[:max_len] + [0]*(max_len-len(i[:max_len])) for i in X]
        self.X = torch.tensor(pad, dtype=torch.int32)
        self.y = torch.Tensor(np.array(list(y)))
# [:, [8, 9, 22, 35]]        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)
    
    
train_ = dataset_(tk_idx_train, lab_t)
# train_ = dataset_(tk_idx_train, train.label)
test_ = dataset_(tk_idx_test, tes)

In [77]:
# class MultAtt(nn.Module):
    
#     def __init__(self, dim_emb=256):
#         super(Att, self).__init__()
#         self.Wq = nn.Linear(dim_emb, dim_emb, bias=False)
#         self.Wk = nn.Linear(dim_emb, dim_emb, bias=False)
#         self.Wv = nn.Linear(dim_emb, dim_emb, bias=False)
#         self.Wo = nn.Linear(dim_emb, 50, bias=False)

#     def forward(self, x):
#         Q = self.Wq(x)
#         K = self.Wk(x)
#         V = self.Wv(x)
#         interval = int(K.size(-1)/4)
#         Z = torch.cat([torch.matmul(nn.Softmax(dim=-1)(torch.matmul(
#                 Q[:,:, i:i+interval], K[:,:, i:i+interval].transpose(-2, -1))
#                 /math.sqrt(interval)), V[:,:, i:i+interval]) 
#                 for i in range(0, K.size(-1), interval)], dim=-1)
#         return self.Wo(Z).transpose(-2, -1)


class Emb(nn.Module):
    
    def __init__(self, voc_size, dim_emb):
        super(Emb, self).__init__()
        self.emb = nn.Sequential(
            nn.Embedding(voc_size, dim_emb, padding_idx=0),
            nn.Linear(dim_emb, 256),
            nn.ReLU())

    def forward(self, x):
        return self.emb(x)

    
class Conv1(nn.Module):
    
    def __init__(self, dim_emb, dim_out):
        super(Conv1, self).__init__()
        self.cv1 = nn.Sequential(
            nn.Conv1d(dim_emb, dim_out, 1),
            nn.BatchNorm1d(dim_out),
            nn.ReLU(),
            nn.MaxPool1d(3, 2))
        self.cv2 = nn.Sequential(
            nn.Conv1d(dim_emb, dim_out, 2),
            nn.BatchNorm1d(dim_out),
            nn.ReLU(),
            nn.MaxPool1d(3, 2)) 
        self.cv3 = nn.Sequential(
            nn.Conv1d(dim_emb, dim_out, 3),
            nn.BatchNorm1d(dim_out),
            nn.ReLU(),
            nn.MaxPool1d(2, 2))

    def forward(self, x):
        C1 = self.cv1(x.transpose(-2, -1)).transpose(-2, -1)
        C2 = self.cv2(x.transpose(-2, -1)).transpose(-2, -1)
        C3 = self.cv3(x.transpose(-2, -1)).transpose(-2, -1)
        return torch.cat([C1,C2,C3], dim=-1)
    
class Conv2(nn.Module):
    
    def __init__(self, dim_emb, dim_out):
        super(Conv2, self).__init__()
        self.cv1 = nn.Sequential(
            nn.Conv1d(dim_emb, dim_out, 1),
            nn.BatchNorm1d(dim_out),
            nn.ReLU(),
            nn.MaxPool1d(4, 2))
        self.cv2 = nn.Sequential(
            nn.Conv1d(dim_emb, dim_out, 2),
            nn.BatchNorm1d(dim_out),
            nn.ReLU(),
            nn.MaxPool1d(3, 2)) 
        self.cv3 = nn.Sequential(
            nn.Conv1d(dim_emb, dim_out, 3),
            nn.BatchNorm1d(dim_out),
            nn.ReLU(),
            nn.MaxPool1d(2, 2))

    def forward(self, x):
        C1 = self.cv1(x.transpose(-2, -1)).transpose(-2, -1)
        C2 = self.cv2(x.transpose(-2, -1)).transpose(-2, -1)
        C3 = self.cv3(x.transpose(-2, -1)).transpose(-2, -1)
        return torch.cat([C1,C2,C3], dim=-1)
    
    
class EndConv(nn.Module):
    
    def __init__(self, dim_emb, dim_out, conv_size):
        super(EndConv, self).__init__()
        self.ecv = nn.Sequential(
            nn.Conv1d(dim_emb, dim_out, conv_size),
            nn.BatchNorm1d(dim_out),
            nn.ReLU())

    def forward(self, x):
        Z = self.ecv(x.transpose(-2, -1))
        return nn.MaxPool1d(Z.size(-1))(Z).squeeze(-1)

    
# class SelfAtt(nn.Module):
    
#     def __init__(self, dim_emb=256, dim_enc=256):
#         super(SelfAtt, self).__init__()
#         self.Wq = nn.Linear(dim_emb, dim_enc, bias=False)
#         self.Wk = nn.Linear(dim_emb, dim_enc, bias=False)
#         self.Wv = nn.Linear(dim_emb, dim_enc, bias=False)

#     def forward(self, x):
#         Q = self.Wq(x)
#         K = self.Wk(x)
#         V = self.Wv(x)
#         score = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(K.size(-1)) ## (50, 1200)
#         return torch.matmul(nn.Softmax(dim=-1)(score), V)
    
            
class Att(nn.Module):
    
    def __init__(self, dim_emb=256, dim_enc=128):
        super(Att, self).__init__()
        self.Wq = nn.Linear(dim_emb, dim_enc, bias=False)
        self.Wk = nn.Linear(dim_emb, dim_emb, bias=False)
        self.Wv = nn.Linear(dim_emb, dim_emb, bias=False)

    def forward(self, x):
        K = self.Wk(x)
        V = self.Wv(x)
        temp = self.Wq(K) 
        score = temp.transpose(-2, -1) / math.sqrt(K.size(-1)) ## (50, 1200)
        return torch.matmul(nn.Softmax(dim=-1)(score), V)
    
    
class LN(nn.Module):
    
    def __init__(self, dim_emb=256):
        super(LN, self).__init__()
        self.feed = nn.LayerNorm(dim_emb)

    def forward(self, x):
        return x + self.feed(x)


class final_model(nn.Module):
    
    def __init__(self, voc_size=len(voc)+1, dim_emb=256, dropout=0.5, max_len=1024):
        super(final_model, self).__init__()
        self.net = nn.Sequential(
            Emb(voc_size, dim_emb),  ## (1024, dim_emb)
            Conv1(dim_emb, 128),
            # Conv2(768, 256),
            nn.Dropout(dropout),
            Att(384, 3),
            nn.Linear(384, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid())
            
    def forward(self, x):
        return self.net(x)

In [78]:
def train_model(model, train_dataset, eval_dataset, device, norm=0.5,
                lr=0.0005, epochs=50, batch_size=256):
    
    history = {'train_loss': [], 'eval_loss': [], 'detail_train': [], 'detail_eval': []}
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

    model = model.to(device)
    save_loss = nn.BCELoss(reduction='none').to(device)
    
    if norm:
        Loss = nn.BCELoss(weight=train_.y.sum(axis=0)**-norm).to(device)

    else:
        Loss = nn.BCELoss().to(device)
        
    op = torch.optim.Adam(model.parameters(), lr=lr)
    
    print('Training start!')
    start = time.time()
    
    for epoch in range(epochs):
        
        model.train()
        train_loss = 0
        eval_loss = 0
        detail_train = torch.zeros(3).to(device)
        detail_eval = torch.zeros(3).to(device)
        pred = []
        real = []
        
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)
            out = model(X).squeeze(-1)
            loss = Loss(out, y)
            save_train = save_loss(out, y).sum(0)
            
            op.zero_grad()
            loss.backward()
            op.step()
            
            train_loss += loss
            detail_train += save_train
            
        model.eval()
        with torch.no_grad():
            for X, y in eval_loader:
                X = X.to(device)
                y = y.to(device)
                out = model(X).squeeze(-1)
                loss = Loss(out, y)
                save_eval = save_loss(out, y).sum(0)
                detail_eval += save_eval
                eval_loss += loss
                pred.append(out.cpu())
                real.append(y.cpu())
                
        train_loss = (train_loss/len(train_loader)).item()
        eval_loss = (eval_loss/len(eval_loader)).item() 
        
        history['train_loss'].append(train_loss)
        history['eval_loss'].append(eval_loss)  
        history['detail_train'].append(detail_train.cpu().detach())
        history['detail_eval'].append(detail_eval.cpu().detach())  
        
        if not (epoch+1)%10:
            print(f"epoch {epoch+1}\ntrain loss: {train_loss}\t\teval loss: {eval_loss}")
        
        if not (epoch+1)%50:
            res = torch.cat(pred)
            tru = torch.cat(real)
            print(f'\nepoch {epoch+1}:\n')
            print(f"f1_score for 50 classes: {f1_score(tru, np.round(res), average='macro')}")
            print(classification_report(tru, np.round(res), target_names=['8','22','35']))
            print(f'\nSpent time: {time.time()-start} seconds')
            
        
    print('Training complete!')
    
    return history

In [79]:
mod = final_model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 200
lr = 3e-4
batch_size = 128
norm = None

hist = train_model(mod, train_, test_, device, norm,
                lr=lr, epochs=epochs, batch_size=batch_size)

plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(hist['train_loss'], label='Train')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(hist['eval_loss'], label='Eval')
plt.legend();

Training start!
epoch 10
train loss: 0.0833403617143631		eval loss: 0.2812250256538391
epoch 20
train loss: 0.02014259621500969		eval loss: 0.4391322731971741
epoch 30
train loss: 0.00949272233992815		eval loss: 0.5297831892967224
epoch 40
train loss: 0.00764800701290369		eval loss: 0.5492178797721863
epoch 50
train loss: 0.010294455103576183		eval loss: 0.5933692455291748

epoch 50:

f1_score for 50 classes: 0.5410353201413768
              precision    recall  f1-score   support

           8       0.64      0.53      0.58       109
          22       0.51      0.58      0.55       216
          35       0.58      0.43      0.49       219

   micro avg       0.56      0.51      0.53       544
   macro avg       0.58      0.51      0.54       544
weighted avg       0.57      0.51      0.53       544
 samples avg       0.11      0.10      0.10       544


Spent time: 474.1661067008972 seconds


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epoch 60
train loss: 0.004924250766634941		eval loss: 0.6368927359580994
epoch 70
train loss: 0.009446466341614723		eval loss: 0.5365355014801025
epoch 80
train loss: 0.0026659886352717876		eval loss: 0.7390288710594177
epoch 90
train loss: 0.003917635418474674		eval loss: 0.7801422476768494
epoch 100
train loss: 0.0024933137465268373		eval loss: 0.7829533219337463

epoch 100:

f1_score for 50 classes: 0.5564513434409813
              precision    recall  f1-score   support

           8       0.53      0.61      0.57       109
          22       0.65      0.56      0.60       216
          35       0.61      0.42      0.50       219

   micro avg       0.60      0.51      0.56       544
   macro avg       0.60      0.53      0.56       544
weighted avg       0.61      0.51      0.55       544
 samples avg       0.11      0.11      0.11       544


Spent time: 974.8765230178833 seconds


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epoch 110
train loss: 0.0024784435518085957		eval loss: 0.8670713305473328


KeyboardInterrupt: 

In [80]:
pd.read_csv('new.csv')

,label_idx,cat_name,freq,word_emb_loss,tfidf_loss
0,0,Accounting and Finance,935,1081.27860,611.02856
1,1,Antitrust,880,755.36646,472.20645
2,2,Banking,1078,848.25390,548.88180
3,3,Broker Dealer,670,633.12213,453.74570
4,4,Commodities Trading,682,476.77130,355.12900
5,5,Compliance Management,1391,937.76760,696.33780
6,6,Consumer protection,969,986.31890,570.24980
7,7,Contract Provisions,1153,999.73500,607.11096
8,8,Corporate Communications,518,810.62720,491.92825
9,9,Corporate Governance,958,1324.72840,706.44617


In [89]:
txt_8 = [i for i in range(len(train)) if 8 in train.label_idx[i]]
txt_22 = [i for i in range(len(train)) if 22 in train.label_idx[i]]
txt_35 = [i for i in range(len(train)) if 35 in train.label_idx[i]]

In [98]:
train.iloc[txt_8]

,id,name,document_text,label_idx,label
11,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,"[16, 23, 46, 44, 18, 8, 13]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
12,4786,ASC Reciprocates Criminal Fraud Conviction Aga...,The Alberta Securities Commission (ASC) has is...,"[8, 23, 13, 46, 44, 16, 18, 6]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ..."
28,4815,Australian Securitisation Forum 2013 The regul...,Australian Securitisation Forum 2013 A speech ...,"[29, 40, 0, 8, 38, 16, 11, 20]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
32,4819,ASIC Corporations (Conditional Costs Schemes) ...,Australian Securities and Investments Commissi...,"[8, 24, 40, 25]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
44,4835,Astivita Writes Down Deferred Tax Asset and Br...,ASIC noted the decision by ASX-listed Astivita...,"[15, 37, 35, 0, 8, 39]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
...,...,...,...,...,...
9758,56806,"Vinco Ventures, Inc. – Distribution - Option S...",The Options Clearing Corporation published tha...,"[11, 7, 47, 49, 8, 46, 48, 34, 43]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."
9759,56807,"Amazon.com, Inc. - 20 For 1 Stock Split - Opti...",The Options Clearing Corporation published tha...,"[7, 34, 8, 43, 11]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."
9809,56973,Information Regarding The Merger Of IRSA Propi...,"On May 20, 2022, BNY Mellon announced a merger...","[14, 8]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
9844,57155,Processing of ASBA Applications in Public Issu...,Securities and Exchange Board of India (SEBI) ...,"[8, 45, 13, 44]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [96]:
ori = pd.read_csv('datasolve-us/train.csv')

In [100]:
ori[ori['id']==4785]

,id,name,document_text,cat_name
4929,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,Financial Crime
9641,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,Legal Proceedings
10872,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,Securities Sales
13487,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,Securities Issuing
24707,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,Fraud
35679,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,Corporate Communications
42448,4785,Decision In the Matter of Marie Louise LaFramb...,Alberta Securities Commission staff (Staff) se...,Exemptions


In [101]:
dic = {i: [] for i in range(50)}

for i in range(50):
    for j in train.label_idx:
        if i in j:
            temp = j[:]
            temp.remove(i)
            dic[i].append(temp)

In [105]:
dic[8]

[[16, 23, 46, 44, 18, 13],
 [23, 13, 46, 44, 16, 18, 6],
 [29, 40, 0, 38, 16, 11, 20],
 [24, 40, 25],
 [15, 37, 35, 0, 39],
 [34, 41, 31, 29, 0],
 [41, 39, 6, 0],
 [29, 28, 22, 13, 47, 46],
 [25, 27, 49, 9, 17, 46, 45, 5, 3, 36, 24, 7, 40, 48, 29, 47, 14],
 [5, 49, 39, 9, 43, 33, 47, 29, 46],
 [37, 33, 43],
 [],
 [12, 40, 2, 20, 5],
 [37, 2],
 [17, 12, 13, 37, 2],
 [2, 23],
 [29, 12, 13],
 [39, 20, 44, 2, 5, 40, 29],
 [18, 16, 2, 6],
 [39, 44, 20, 12],
 [24, 7, 22, 1, 23, 47, 6, 28, 5],
 [41, 0, 40],
 [21, 14, 7],
 [20, 15, 35, 39, 0, 44, 46],
 [35, 20, 15, 39, 0],
 [14, 1, 28, 46, 16, 7],
 [14, 39, 46],
 [12, 14, 28, 1, 16, 20],
 [12, 44, 19, 5, 15, 17],
 [24, 16, 15, 44, 18, 3, 0, 22, 25, 26],
 [44, 3, 15, 39, 40, 20, 14, 0],
 [14, 3, 44, 20, 39, 40, 0, 15],
 [33, 5, 15, 44, 0, 39],
 [16, 28, 5, 12, 24, 46, 18, 7, 23, 10, 13, 25, 38],
 [43, 48, 47, 44, 13],
 [43, 48, 44, 47, 13],
 [9, 42, 43, 29, 34, 33, 45],
 [20, 44],
 [44, 12, 23, 20, 46, 13, 22],
 [26, 24, 13, 46, 10, 27, 44, 29,

In [106]:
save = []
for x in [8, 22, 35]:
    mb = []
    for i in range(50):
        dom = 0
        non_dom = 0
        count_1 = 0
        count_2 = 0
        nc_1 = 0
        nc_2 = 0
        if i not in [8, 22, 35]:
            for j in train.label_idx:
                if i in j:
                    dom += 1
                    if x in j:
                        count_1 += 1
                    else:
                        nc_1 += 1
                else:
                    non_dom += 1
                    if x in j:
                        count_2 += 1
                    else:
                        nc_2 += 1
            mb.append((count_1/dom, count_2/non_dom, nc_1/dom, nc_2/non_dom))
    save.append(mb)

In [110]:
save[0]

[(0.051336898395721926,
  0.052666965486329,
  0.948663101604278,
  0.947333034513671),
 (0.025, 0.05524000445483907, 0.975, 0.9447599955451609),
 (0.021335807050092765,
  0.05637171165015374,
  0.9786641929499073,
  0.9436282883498462),
 (0.13880597014925372,
  0.046250952225486995,
  0.8611940298507462,
  0.953749047774513),
 (0.005865102639296188,
  0.05600958919036722,
  0.9941348973607038,
  0.9439904108096328),
 (0.04385334291876348,
  0.05396787907416155,
  0.9561466570812365,
  0.9460321209258384),
 (0.0392156862745098,
  0.05399325084364454,
  0.9607843137254902,
  0.9460067491563554),
 (0.08065915004336513,
  0.04881690787962325,
  0.9193408499566349,
  0.9511830921203768),
 (0.03653444676409186,
  0.05426356589147287,
  0.9634655532359081,
  0.9457364341085271),
 (0.03684210526315789,
  0.053504144687264506,
  0.9631578947368421,
  0.9464958553127355),
 (0.13398294762484775,
  0.04514273069263111,
  0.8660170523751523,
  0.9548572693073689),
 (0.029850746268656716,
  0.05741

In [114]:
save[1][21]

(0.22933730454207, 0.07033818694222639, 0.77066269545793, 0.9296618130577736)

In [113]:
pd.read_csv('new.csv')

,label_idx,cat_name,freq,word_emb_loss,tfidf_loss
0,0,Accounting and Finance,935,1081.27860,611.02856
1,1,Antitrust,880,755.36646,472.20645
2,2,Banking,1078,848.25390,548.88180
3,3,Broker Dealer,670,633.12213,453.74570
4,4,Commodities Trading,682,476.77130,355.12900
5,5,Compliance Management,1391,937.76760,696.33780
6,6,Consumer protection,969,986.31890,570.24980
7,7,Contract Provisions,1153,999.73500,607.11096
8,8,Corporate Communications,518,810.62720,491.92825
9,9,Corporate Governance,958,1324.72840,706.44617
